# Libraries and Functions

In [81]:
jupyter notebook --NotebookApp.iopub_data_rate_limit=100000000
import geopandas
#import networkx as nx
#import osmnx as ox
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
%matplotlib inline

# Basic Data Reading and Analysis

In [82]:
rdata= pd.read_excel(r'C:\Users\Rachael\Downloads\romeroads.xlsx', dtype={'Source':str, 'Destination': str, 'Weigth':int})
print(rdata.head())

#How many unique destination and source verticess exist?
unique_sources=rdata.Source.unique()
unique_destinations=rdata.Destination.unique()
print(len(unique_sources))
print(len(unique_destinations))

#Is the set of unique sources and destinations similar?
print(set(unique_sources)==set(unique_destinations))

  Source Destination  Weight
0   2959        2958     535
1   1573        1494      77
2   1603        1590     184
3   2635        2634     179
4   1985        1984      22
3353
3353
True


# Adjacency Matrix and Dijkstra Algorithm

In [186]:
##March##

#Vertex object to hold the road network data whose indices will correspond row / column numbers in the adjacency matrix
class Vertex:
    def __init__(self, data, indexloc = None):
        self.data = data
        self.index = indexloc
        
        
        
class Graph:

 #a helper method to give flexibility in using either a vertex's index or
 #vertex object as arguments in the class Graph
    @classmethod
    
    
    
    
 #create a graph with dimensions equal to the no. of vertices
    def graph_shape(self, vertices):
        return Graph(len(vertices), len(vertices), vertices)  
    

    
    
 
 #Create the  Adjacency Matrix
    def __init__(self, rows, cols, vertices = None):
        self.mat = [[0] * cols for _ in range(rows)]
        self.vertices = vertices
        for i in range(len(self.vertices)):
            self.vertices[i].index = i

            
            
 #create the notion of connectes/linked vertices plus their weights
 #The rows are viewed as sources while the columns as my destinations
    def vertlink(self, vertex1,vertex2, weight = 1):
        vertex1=self.get_index(vertex1)
        vertex2=self.get_index(vertex2)
        self.mat[vertex1][vertex2] = weight
        
        
        
  
 #weights included 
    def link(self, vertex1,vertex2, weight = 1):
        self.vertlink(vertex1, vertex2, weight)
        self.vertlink(vertex2, vertex1, weight)
        
        
            
   #For a row, traverse across it horizontally finding links to columns
   # to the row vertex (origin) and the weights  
   #It returns an array that holds a tuple of vertices and weights 
    def originvert(self, vertex):
        vertex=self.get_index(vertex)
        return [(self.vertices[col_num],self.mat[vertex][col_num]) for col_num in range(len(self.mat[vertex])) if self.mat[vertex][col_num] != 0]

   
    #To define our verices
    def vertex(self, index):
        return self.vertices[index]


   #For a column,traverse it vertically finding rows that 
    #are connected to the column vertex (target) and the weight 
    def targetvert(self, vertex):
        vertex=self.get_index(vertex)
        column=[row[vertex] for row in self.mat]
        return [(self.vertices[row_num], column[row_num]) for row_num in range(len(column)) if column[row_num] != 0]

  
 
     
 #we can anticipate getting rid of links between vertices should the need arise
 #wwhile allowing both indices and node objects
    
    def drop_link(self, vertex1, vertex2):
        self.drop_vert_link(vertex1, vertex2)
        self.drop_vert_link(vertex2, vertex1)
        
    def drop_vert_link(self, vertex1, vertex2):
        vertex1=self.get_index(vertex1)
        vertex2=self.get_index(vertex2)
        self.mat[vertex1][vertex2] = 0 
        
        
        
            
   #To allow addition of vertices to our graph hence row in our matrix
    def newver(self,vertex):
        self.vertices.append(vertex)
        vertex.index = len(self.vertices) - 1
        for row in self.mat:
            row.append(0)     
        self.mat.append([0] * (len(self.adj_mat) + 1))
    
    
   # Should the need to enquire whether we can bridge from one vertex to another arise, we can:   
    def bridge(self, vertex1, vertex2):
        vertex1 = self.get_index(vertex1)
        vertex2 = self.get_index(vertex2)
        return self.mat[vertex1][vertex2] != 0  
  
    def link_exists(self, vertex1, vertex2):
        return self.bridge(vertex1, vertex2) or self.bridge(vertex2, vertex1)

        
 #To get the  weights   
    def edge_val(self, n1, n2):
        vertex1 = self.get_index(n1)
        vertex2 = self.get_index(n2)
        return self.adj_mat[vertex1][vertex2]
  

 #The predominant get_index function that either returns vertices or indices 
    def get_index(self, vertex):
        if not isinstance(vertex, Vertex) and not isinstance(vertex, int):
            raise ValueError("vertex must be an integer or a Vertex object")
        if isinstance(vertex, int):
            return vertex
        else:
            return vertex.index
        
      
    def printmat(self):
        for row in self.mat:
            print(row)   
            
        
 #DIJKSTRA


    # As described in the algorithm
    #
    def dijkstra(self, vertex):
            vertexind = self.get_index(vertex) #retrieve the index of a vertex 
            road_stretch = [None] * len(self.vertices)
            for i in range(len(road_stretch)):
                road_stretch[i] = [float("inf")] #initialize to infinity the temporary stretch from the source to all other vertices
                road_stretch[i].append([self.vertices[vertexind]])

            road_stretch[vertexind][0] = 0
            #Fstore vertices in a queue
            queue = [i for i in range(len(self.vertices))] #queue all vertices in the graph
            visited = set()
            while len(queue) > 0:
                minimum_road_stretch = float("inf") #not yet visited
                minimum_vertex = None        
                for n in queue:         #with the smallest distance
                    if road_stretch[n][0] < minimum_road_stretch and n not in visited:
                        minimum_road_stretch = road_stretch[n][0]
                        minimum_vertex = n
                queue.remove(minimum_vertex)
                visited.add(minimum_vertex) 
                connections = self.originvert(minimum_vertex)
                for (vertex, weight) in connections: 
                    tot_stretch = weight + minimum_road_stretch
                    if tot_stretch < road_stretch[vertex.index][0]:
                        road_stretch[vertex.index][0] = tot_stretch
                        road_stretch[vertex.index][1] = list(road_stretch[minimum_vertex][1])
                        road_stretch[vertex.index][1].append(vertex)
            return road_stretch 

        
        
#Applying to our data        
unique_sources=rdata.Source.unique()

for i in range(len(unique_sources)):
    unique_sources[i]=Vertex(unique_sources[i]) 

loc_data=rdata.replace(rdata.Source.unique().tolist(),unique_sources)

graph = Graph.graph_shape(unique_sources)

for  i in range(len(loc_data)):
     graph.link(loc_data.iloc[i,0],loc_data.iloc[i,1],loc_data.iloc[i,2])
        
#graph.printmat() #Should you want to print out the adjacency matrix

print([(weight, [n.data for n in vertex]) for (weight, vertex) in graph.dijkstra(loc_data.iloc[0,0])])        

[(0, ['2959']), (25559, ['2959', '2958', '2953', '2947', '2946', '2928', '2925', '2875', '2871', '2872', '2876', '2832', '2827', '2818', '2816', '2815', '2804', '2803', '2628', '2623', '2624', '2615', '2587', '2581', '2574', '2573', '2572', '2404', '2402', '2385', '2376', '2348', '2346', '2344', '2329', '1887', '1886', '1882', '1940', '1938', '1914', '1913', '1908', '1915', '1916', '1900', '1901', '1805', '1802', '1789', '1792', '1791', '1785', '1786', '1752', '1751', '1753', '1748', '1968', '1967', '1605', '1598', '1565', '1566', '1572', '1569', '1573']), (24340, ['2959', '2958', '2953', '2954', '2943', '2942', '2940', '2933', '2891', '2855', '2854', '2847', '2840', '2833', '2621', '2619', '2592', '2588', '2580', '2409', '2411', '2407', '2400', '2397', '2396', '2395', '2364', '2361', '2359', '2358', '2360', '2422', '2057', '2056', '1966', '2055', '2052', '2051', '2050', '2032', '2033', '2010', '2009', '2008', '2007', '2000', '1999', '1985', '1984', '1972', '1970', '1603']), (18284, ['

# Dijkistra with Binary Heaps

In [184]:
class Vertex:
  
    def __init__(self, data, indexloc = None):
        self.data = data
        self.index = indexloc

        
class BinaryTree:

    def __init__(self, vertices = []):
        self.vertices = vertices

    def root(self):                   #root is assigned index 0
        return self.vertices[0]
    
    def parent_index(self, i):         # floor((i-1)/2) index for parent vertex i
        return (i - 1) // 2
    
    def left_index(self, i):           #left children will  have odd indices
        return 2*i + 1
    
    def right_index(self, i):          #right children will  have even indices
        return 2*i + 2

    def left(self, i):
        return self.vertex_index(self.left_index(i))
    
    def right(self, i):
        return self.vertex_index(self.right_index(i))

    def parent(self, i):
        return self.vertex_index(self.parent_index(i))

    def vertex_at_index(self, i):
        return self.vertices[i]

    def size(self):                           #number of vertices
        return len(self.vertices)


    
 


        
#We have to turn binary tree into a minimum heap, pop minimum value, reorganize the
# the heap and update values of vertices to keep it as a heap. The
# function is called at every vertex i. First case assumption is that 
#the subtrees satisfy the heap property.

#lambda is an anonymous function for comparison and return an updated 
#vertex

class MinimumHeap(BinaryTree):

    def __init__(self, vertices, less_than = lambda a,b: a < b, get_index = None, update_vertex = lambda vertex, newval: newval):
        BinaryTree.__init__(self, vertices)
        self.mapping = list(range(len(vertices)))
        self.less_than, self.get_index, self.update_vertex = less_than, get_index, update_vertex
        self.min_heapify()
        
    #Assumes the entire heap is accurate save for a single 3-node
    # subtree below. It will used recursively by finding parent node i
    #check if the children are smaller than it (violation of min heap proerty)
    #If they are, we swap the smallest child with parent and
    # process repeated recursively to the point where the heap
    def subtree_in_minimum_heap(self, i):

        size = self.size()
        left_index = self.left_index(i)
        right_index = self.right_index(i)
        minimum_index = i
        if( left_index < size and self.less_than(self.vertices[left_index], self.vertices[minimum_index])):  #comparison
            minimum_index = left_index
        if( right_index < size and self.less_than(self.vertices[right_index], self.vertices[minimum_index])): #comparison
            minimum_index = right_index
        if( minimum_index != i):
            self.vertices[i] = self.vertices[minimum_index]                 #swapping 
            self.vertices[minimum_index] = self.vertices[i]                 #swapping
                
            if self.get_index is not None:
                self.mapping[self.get_index(self.vertices[minimum_index])] = minimum_index
                self.mapping[self.get_index(self.vertices[i])] = i
            self.subtree_in_minimum_heap(minimum_index)


#Another case scenario is when the array is entirely random
#and we now have to run through its entirety. Therefore
#recursively call subtree_in_minimum. The minimum will be the root, 0
    def min_heapify(self):
        for i in range(len(self.vertices), -1, -1):
            self.subtree_in_minimum_heap(i)

    def minimum(self):
        return self.vertices[0]
#To remove the minimum, pop it off and replace with last leaf
#and call subtree_in_minimum
    def pop(self):
        minimum = self.vertices[0]
        if self.size() > 1:
            self.vertices[0] = self.vertices[-1]
            self.vertices.pop()
            if self.get_index is not None:
                self.mapping[self.get_index(self.vertices[0])] = 0
            self.subtree_in_minimum_heap(0)
        elif self.size() == 1: 
            self.vertices.pop()
        else:
            return None
       
        if self.get_index is not None:
            self.mapping[self.get_index(minimum)] = None
        return minimum

    # Update vertex value, bubble it up as necessary to maintain heap
    #property
    def decrease_key(self, i, val):
        self.vertices[i] = self.update_vertex(self.vertices[i], val)
        parent_index = self.parent_index(i)
        while( i != 0 and not self.less_than(self.vertices[parent_index], self.vertices[i])):
            self.vertices[parent_index]= self.vertices[i],
            self.vertices[i] = self.vertices[parent_index]
            # If there is a Psi to get absolute index of a vertex
            # update your mapping array to indicate where that index lives
            # in the vertices array (so lookup by this index is O(1))
            if self.get_index is not None:
                self.mapping[self.get_index(self.vertices[parent_index])] = parent_index
                self.mapping[self.get_index(self.vertices[i])] = i
            i = parent_index
            parent_index = self.parent_index(i) if i > 0 else None

    def index_of_vertex_at(self, i):
        return self.get_index(self.vertices[i])
    
    
    
    
#provide extra information on provisional distance and the list of 
#jumps  made
class decorator:
    
    def __init__(self, vertex):
        self.vertex = vertex
        self.provisional_distance = float('inf')
        self.jumps = []

    def index(self):
        return self.vertex.index

    def data(self):
        return self.verttex.data
   #Allow update of object 
    def update_data(self, data):
        self.provisional_distance = data['provisional_distance']
        self.jumps = data['jumps']
        return self
    

    
    
    
#TO KEEP A RELATIVELY SIMILAR INTERFACE AS THE ADJACENCY MATRIX BIT
#ABOVE BUT WHILE USING ADJACENCY LISTS RATHER THAN MATRICES
class Graph: 

    def __init__(self, vertices):
        self.adjacency_list = [ [vertex, []] for vertex in vertices ]
        for i in range(len(vertices)):
            vertices[i].index = i

            
            
            
   #Each vertex will have corresponding vertices that represent
#links (non-zero weights) thus simplifying iterations
   
    def get_index_from_vertex(self, vertex):
        if not isinstance(vertex, Vertex) and not isinstance(vertex, int):
            raise ValueError("vertex must be an integer or a vertex object")
        if isinstance(vertex, int):
            return vertex
        else:
            return vertex.index
        
          

    def vertlink(self, vertex1, vertex2, weight = 1):
        vertex1 = self.get_index_from_vertex(vertex1) 
        vertex2 = self.get_index_from_vertex(vertex2)

        self.adjacency_list[vertex1][1].append((vertex2, weight))

    def link(self, vertex1, vertex2, weight = 1):
        self.vertlink(vertex1, vertex2, weight)
        self.vertlink(vertex2, vertex1, weight)

    
    def links(self, vertex):
        vertex = self.get_index_from_vertex(vertex)
        return self.adjacency_list[vertex][1]
    
      
    

    def dijkstra(self, source):
        
        index_of_source = self.get_index_from_vertex(source)
        destination_vertices = [decorator(vertex_edges[0]) for vertex_edges in self.adjacency_list]
        destination_vertices[index_of_source].provisional_distance = 0
        destination_vertices[index_of_source].jumps.append(destination_vertices[index_of_source].vertex) 
        less_than = lambda a, b: a.provisional_distance < b.provisional_distance
        get_index = lambda vertex: vertex.index()
        update_vertex = lambda vertex, data: vertex.update_data(data)

    
        heap = MinimumHeap(destination_vertices, less_than, get_index, update_vertex)
        min_dist_list = []

        while heap.size() > 0:
            min_decorated_vertex = heap.pop()
            min_dist = min_decorated_vertex.provisional_distance
            jumps = min_decorated_vertex.jumps
            min_dist_list.append([min_dist, jumps])
            
            
            links = self.links(min_decorated_vertex.vertex)
            for (ivertex, weight) in links: 
                vertex = self.adjacency_list[ivertex][0]
                heap_location = heap.mapping[ivertex]
                if(heap_location is not None):
                    tot_dist = weight + min_dist
                    if tot_dist < heap.vertices[heap_location].provisional_distance:
                        jumps_cpy = list(jumps)
                        jumps_cpy.append(vertex)
                        data = {'provisional_distance': tot_dist, 'jumps': jumps_cpy}
                        heap.decrease_key(heap_location, data)

        return min_dist_list 

#  FIBONACCI HEAP 

In [187]:
class FibonacciHeap:

    #Vertex class to implement vertices of a fibonacci heap and
    #their characteristics of interest.
    
    class Vertex:
        def __init__(self, data):
            self.data = data
        def __init__(self, key, value):
            self.key    = key
            self.value  = value
            self.degree = 0  #the number of connections that a vertex has to other vertices in the network.
            self.mark   = False  # number of marked nodes in your heap
            self.parent = None
            self.child  = None
            self.leftchild   = None
            self.rightchild  = None
            
                
    def find_minimum(self):
        return self.minimum_vertex

    
    #other xtics may include the number of trees in heap and number of marked 
    # vertice you'll calculate from 'self.mark'
            
    #To perform inserion, create a new tree, containing the new value(s)
    #add_vertex into root_list
    #Then update the minimum pointer where applicable and total number of vertices.
    
    def add_vertex(self, key, value=None):
        new = self.Vertex(key, value) 
        new.leftchild  = new
        new.rightchild = new
        self.meld_into_root_list(new)
        if self.minimum_vertex is None or new.key < self.minimum_vertex.key:
            self.minimum_vertex = new
        self.TotalElements += 1
        return new        
    
    
    
 #To define the function meld_into_root list called above
    
    def meld_into_root_list(self, vertex):
        if self.root_list is None:
            self.root_list = vertex
        else:
            vertex.rightchild = self.root_list.rightchild
            vertex.leftchild = self.root_list
            
            self.root_list.rightchild.leftchild = vertex
            self.root_list.rightchild = vertex
            
                   
            
    # Because our iteration will be through a doubly linked list
    def iterate(self, head):
        vertex= stop = head
        flag = False
        while True:
            if vertex == stop and flag is True:
                break
            elif vertex == stop:
                flag = True
                
            yield vertex  
            vertex = vertex.rightchild
   # initialize our root list, minimum and total number of vertices.
    root_list  = None
    min_vertex = None
    TotalElements = 0
    
       
   #to remove a node from the doubly linked root list
        
    def remove_from_root_list(self, vertex):
        if self.root_list is None:
            print('Empty')
            
        if vertex == self.root_list:
            self.root_list = vertex.rightchild
        vertex.leftchild.rightchild = vertex.rightchild
        vertex.rightchild.leftchild = vertex.leftchild
        
          

    #To ensure that no root_list elements have the same degree 
    #during this process we traverse the root_list
    #merge the trees with the same degree
    #use an array that contains for every degree value d a pointer 
    #to a tree left of the current pointer whose root has degree d (if it exists)
    
    def consolidate(self):
        Ranks = [None] * int(math.log(self.TotalElements) * 2)
        #Think of this idea below too
        #Ranks=[None]*self.TotalElement *2
        vertices = [j for j in self.iterate(self.root_list)]
        for j in range(0, len(vertices)):
            x = vertices[j]
            deg = x.degree
            while Ranks[deg] != None:
                y = Ranks[deg]
                if x.key > y.key:
                    temporary = x
                    x= y
                    y = temporary
                self.heap_link(y, x)
                Ranks[deg] = None
                deg += 1
            Ranks[deg] = x
        
        for i in range(0, len(Ranks)):
            if Ranks[i] is not None:
                if Ranks[i].key < self.min_vertex.key:
                    self.min_vertex = Ranks[i]
                    
                                    
    
    
    #To join two trees that may have the same degree in the unit root during
    #consolidation, then increase the degree of the new parent 

    def heap_link(self, y, x):
        self.remove_from_root_list(y)
        y.leftchild = y.rightchild = y
        self.merge_with_child_list(x, y)
        x.degree += 1
        y.parent = x
        y.mark = False
        
        
   #shift one tree to another to become a child so as no two
   # vertices in  the root_list have equal degrees
                            
    def merge_with_child_list(self, parent, vertex):
        if parent.child is None:
            parent.child = vertex
        else:
            Vertex.rightchild = parent.child.rightchild
            Vertex.leftchild = parent.child
            parent.child.rightchild.leftchild = vertex
            parent.child.rightchild = vertex      
                                  
                    
    def merge(self, h2):
        H = FibonacciHeap()
        H.root_list=self.root_list #root list of main heap H
        H.min_vertex =self.min_vertex #minimum of main heap H
        last = h2.root_list.leftchild
        h2.root_list.leftchild = H.root_list.leftchild
        H.root_list.leftchild.rightchild = h2.root_list
        H.root_list.leftchild = last
        H.root_list.leftchild.rightchild = H.root_list
        if h2.min_vertex.key < H.min_vertex.key:
            H.min_vertex = h2.min_vertex
        H.total_vertexs = self.TotalElements + h2.TotalElements
        return H   
    
            
    #Identify the minimum
    #check if it has children. If they exist, correctly meld them into the root_list
    #Then remove the minimum and update the minimum pointer
    #In order to extract and/or delete the minimum vertex 
    #from the heap in O(log n)
    def extract_min(self):
        minimum= self.min_vertex
        if minimum is not None: #Identify if an existing minimum vertex has children
            if minimum.child is not None:
                children = [j for j in self.iterate(minimum.child)] # add them to root_list
                for i in range(0, len(children)):
                    self.meld_into_root_list(children[i])
                    children[i].parent = None   
            self.remove_from_root_list(minimum)
            # set new min Vertex in heap
            if minimum== minimum.rightchild:
                self.min_vertex = self.root_list = None
            else:
                self.min_vertex = minimum.rightchild
                self.consolidate()
            self.TotalElements -= 1
        return minimum

   
    # To ensure we don't violate the heap property apply decrease_key notion
    # case 1: heap property not violated and we just decrease key-vale
    #case 2: heap property violated but parent not marked
    #case 3: heap property violated and parent is marked
     
    def decrease_key(self, x, k):
        if k > x.key:
            return None
        x.key = k
        y = x.parent
        if y is not None and x.key < y.key:
            self.cut(x, y)
            self.cascading_cut(y)
        if x.key < self.min_vertex.key:
            self.min_vertex = x
            
                       
    #Case 2 and 3 scenario:
    # if a child Vertex becomes smaller than its parent Vertex we
    # cut this child Vertex off and bring it up to the root list 
    # and adjust minimum pointers if necessary
    #mark the previous parent UNLESS it is a root
    
    def cut(self, x, y):
        self.remove_from_child_list(y, x)
        y.degree -= 1
        self.meld_into_root_list(x)
        x.parent = None
        x.mark = False
   

    #to remove the child violating the heap property    
    def remove_from_child_list(self, parent, vertex):
        if parent.child == parent.child.rightchild:
            parent.child = None
        elif parent.child == vertex:
            parent.child = vertex.rightchild
            vertex.rightchild.parent = parent
        vertex.leftchild.rightchild = vertex.rightchild
        vertex.rightchild.leftchild = vertex.leftchild    
        
        
   #When parents are marked, cut and make the bit violating heap property
   #into a root, adjust minimum pointers where necessary and continue
   #cutting the parent until you have reached an unmarked vertex. 
   
    def cascading_cut(self, y):
        minimum= y.parent
        if minimum is not None:
            if y.mark is False:
                y.mark = True
            else:
                self.cut(y, z)
                self.cascading_cut(z)
                
                                
                
#TO KEEP A RELATIVELY SIMILAR INTERFACE AS THE ADJACENCY MATRIX BIT
#ABOVE BUT WHILE USING ADJACENCY LISTS RATHER THAN MATRICES
class Graph: 

    def __init__(self, vertices):
        self.adjacency_list = [ [vertex, []] for vertex in vertices ]
        for i in range(len(vertices)):
            vertices[i].index = i

            
            
            
   #Each vertex will have corresponding vertices that represent
#links (non-zero weights) thus simplifying iterations
   
    def get_index_from_vertex(self, vertex):
        if not isinstance(vertex, Vertex) and not isinstance(vertex, int):
            raise ValueError("vertex must be an integer or a vertex object")
        if isinstance(vertex, int):
            return vertex
        else:
            return vertex.index
        
          

    def vertlink(self, vertex1, vertex2, weight = 1):
        vertex1 = self.get_index_from_vertex(vertex1) 
        vertex2 = self.get_index_from_vertex(vertex2)

        self.adjacency_list[vertex1][1].append((vertex2, weight))

    def link(self, vertex1, vertex2, weight = 1):
        self.vertlink(vertex1, vertex2, weight)
        self.vertlink(vertex2, vertex1, weight)

    
    def links(self, vertex):
        vertex = self.get_index_from_vertex(vertex)
        return self.adjacency_list[vertex][1]
    
                                 
    
def dijkstra(source, adjacency_list, sink = None):
    n = len(adjacency_list)   
    distance = [float('inf')]*n

    vertices_of_heap = [None]*n
    heap = FibonacciHeap()
    for i in range(1, n):
        vertices_of_heap[i] = heap.add_vertex(float('inf'), i)     # distance, label

    distance[source] = 0
    heap.decrease_key(vertices_of_heap[source], 0)

    while heap.TotalElements:
        current = heap.extract_min().value
        visited[current] = True

        #early exit
        if sink and current == sink:
            break

        for (neighbor, cost) in adjacency_list[current]:
            if not visited[neighbor]:
                if distance[current] + cost < distance[neighbor]:
                    distance[neighbor] = distance[current] + cost
                    heap.decrease_key(vertices_of_heap[neighbor], distance[neighbor])


    return distance               
                

In [ ]:
def generate_output(input_file):

    the_graph = read_edges_file(input_file)
    distance_list = dijkstra(the_graph)
    output = []
    for i in range(1, len(distance_list)):
        output.append(str(str(i) + " " + str(distance_list[i])))
    return output


if __name__ == '__main__':
    txt_file = sys.argv[1]
    output = generate_output(txt_file)
    f = open("output_dijkstra.txt", "w+")
    for item in output:
        f.write(str(item) + '\n')
    f.close()